# Yellow Taxi Data performance - Part 6

In [5]:
import os
VASTDB_NYT_BUCKET = os.getenv('VASTDB_NYT_BUCKET')
VASTDB_NYT_SCHEMA = os.getenv('VASTDB_NYT_SCHEMA')
VASTDB_NYT_TABLE = os.getenv('VASTDB_NYT_TABLE')

print(VASTDB_NYT_BUCKET, VASTDB_NYT_SCHEMA, VASTDB_NYT_TABLE)

S3A_BUCKET = os.getenv('S3A_BUCKET')
S3_HIVE_TAXI_URI = os.getenv("S3A_HIVE_TAXI_URI")

print(S3_HIVE_TAXI_URI)

csnow-db nyt taxi
s3a://csnow-bucket/nyt/


## VastDB Filter Select

In [18]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%load_ext sql
%config SqlMagic.displaylimit = None

%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/vast  --connection_arguments '{"http_scheme":"https", "verify": false}'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [29]:
%%sql

select 
    *
from
    vast."{{VASTDB_NYT_BUCKET}}|{{VASTDB_NYT_SCHEMA}}".{{VASTDB_NYT_TABLE}}
where
    trip_distance > 129.5 and trip_distance < 130.0

Running query in 'trino://admin@10.143.11.241:8443/vast'

airport_fee,congestion_surcharge,dolocationid,dropoff_latitude,dropoff_longitude,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,pickup_latitude,pickup_longitude,ratecodeid,store_and_fwd_flag,surcharge,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid
None,None,265,-999.0,-999.0,0.0,300.0,0.3,0.0,3,2,230,-999.0,-999.0,5,-999.0,-999.0,0.0,0.0,300.3,-999,-999,129.7,-999
None,None,265,-999.0,-999.0,0.0,400.0,0.3,0.5,1,1,48,-999.0,-999.0,5,-999.0,-999.0,0.0,0.0,400.8,-999,-999,129.96,-999
None,None,265,-999.0,-999.0,1.0,331.0,0.3,0.5,1,2,265,-999.0,-999.0,1,-999.0,-999.0,0.0,5.76,338.56,-999,-999,129.89,-999
-999.0,-999.0,-999,40.688759,-74.179871,-999.0,0.0,-999.0,0.0,1,NA,-999,40.713174,-74.009342,1,-999.0,0.0,0.0,0.0,0.0,2010-02-23 09:54:31,2010-02-23 09:32:46,129.9,CMT
1.25,0.0,265,-999.0,-999.0,1.0,337.0,0.3,0.0,1,1,132,-999.0,-999.0,1,-999.0,-999.0,25.0,6.55,371.1,-999,-999,129.53,-999
0.0,2.5,229,-999.0,-999.0,3.0,14.0,0.3,0.5,1,1,151,-999.0,-999.0,1,-999.0,-999.0,3.55,0.0,21.35,-999,-999,129.7,-999
None,None,92,-999.0,-999.0,0.5,2.5,0.0,0.5,1,1,229,-999.0,-999.0,1,-999.0,-999.0,9.62,35.0,48.12,-999,-999,129.7,-999
None,None,265,-999.0,-999.0,0.0,85.0,0.0,0.0,1,2,164,-999.0,-999.0,5,-999.0,-999.0,0.0,0.0,85.0,-999,-999,129.9,-999
None,None,265,-999.0,-999.0,0.0,400.0,0.3,0.5,1,2,249,-999.0,-999.0,5,-999.0,-999.0,0.0,10.5,411.3,-999,-999,129.87,-999
None,0.0,265,-999.0,-999.0,0.0,345.0,0.3,0.0,1,2,48,-999.0,-999.0,3,-999.0,-999.0,0.0,10.5,355.8,-999,-999,129.9,-999


## Hive Row Count

In [13]:
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/hive  --connection_arguments '{"http_scheme":"https", "verify": false}'

In [31]:
%%sql
    
SELECT 
    *
FROM 
    hive.nyt.yellow_tripdata
where
    trip_distance > 129.5 and trip_distance < 130.0

Running query in 'trino://admin@10.143.11.241:8443/vast'

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,dropoff_latitude,dropoff_longitude,pickup_latitude,pickup_longitude,surcharge
-999,-999,-999,1,129.89,1,-999.0,265,265,2,331.0,1.0,0.5,0.0,5.76,0.3,338.56,None,None,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,1,129.96,5,-999.0,48,265,1,400.0,0.0,0.5,0.0,0.0,0.3,400.8,None,None,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,1,129.75,5,-999.0,10,265,2,1.0,0.0,0.0,0.0,11.08,0.3,12.38,None,None,-999.0,-999.0,-999.0,-999.0,-999.0
CMT,2010-02-23 09:32:46,2010-02-23 09:54:31,1,129.9,1,-999.0,-999,-999,NA,0.0,-999.0,0.0,0.0,0.0,-999.0,0.0,-999.0,-999.0,40.688759,-74.179871,40.713174,-74.009342,0.0
CMT,2010-03-18 23:09:29,2010-03-18 23:38:13,1,129.6,1,-999.0,-999,-999,Cas,0.0,-999.0,0.0,0.0,0.0,-999.0,0.0,-999.0,-999.0,40.776984,-74.015366,40.773861,-73.871001,0.0
-999,-999,-999,3,129.7,5,-999.0,230,265,2,300.0,0.0,0.0,0.0,0.0,0.3,300.3,None,None,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,1,129.53,1,-999.0,132,265,1,337.0,1.0,0.0,25.0,6.55,0.3,371.1,0.0,1.25,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,3,129.9,1,-999.0,13,231,1,16.5,3.5,0.5,4.15,0.0,0.3,24.95,2.5,0.0,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,1,129.9,5,-999.0,68,265,2,300.0,0.0,0.0,0.0,13.75,0.3,316.55,2.5,0.0,-999.0,-999.0,-999.0,-999.0,-999.0
-999,-999,-999,1,129.9,1,-999.0,132,265,1,475.5,1.75,0.5,10.0,47.02,1.0,535.77,0.0,1.75,-999.0,-999.0,-999.0,-999.0,-999.0
